# Alternating Least Square (ALS)

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from interaction_table import ncf_clicks_weigher, ncf_orders_weigher, orders_weigher_sum, InteractionTable
from process_data import preprocess_orders_and_clicks, additional_filtration_orders_and_clicks
from h3_index import H3Index

In [2]:
#!pip install fastparquet
h3index = H3Index('../data/h3_to_chains.pkl')

In [3]:
# !mkdir -p ../data/moscow_slice
# preprocess_orders_and_clicks(
#     path_to_orders="../data/orders",
#     path_to_clicks="../data/clicks",
#     save_path="../data/moscow_slice",
# )

In [32]:
def get_clicks():
    path = '../data/clicks/'
    clicks = pd.read_parquet(f'{path}/clicks.parquet')
    return clicks

def get_orders():
    path = '../data/moscow_slice/'
    orders = pd.read_parquet(f'{path}/orders.parquet')
    orders = orders.rename(columns={"customer_id": "user_id"})
    clicks = pd.read_parquet(f'{path}/clicks.parquet')
    #regs = pd.read_pickle('../data/CITIES_MAPPING.pkl')
    #regs = [v for k, v in regs.items() if v > 2]
    regs = [1] # moscow
    orders, _ = additional_filtration_orders_and_clicks(orders, clicks, regs_to_filter=regs)
    return orders

In [33]:
interactions = InteractionTable(None, get_orders,
                                None, ncf_orders_weigher,
                                alpha=0)

Orders df loaded: size=5452738,  uniq_users=1394011,  uniq_chains=7269
Orders df weighted: size=5452738, uniq_users=1394011, uniq_chains=7269


In [34]:
interactions.sparse_interaction_matrix

<7269x1394011 sparse matrix of type '<class 'numpy.int64'>'
	with 3105842 stored elements in Compressed Sparse Row format>

In [39]:
test = interactions.interaction_df[['user_id', 'weight']]
test = test.groupby('user_id').sum()
test = test.reset_index()[['user_id', 'weight']]
user_with_few_interactions = set(test[test['weight'] <= 5].user_id.unique())
ncf_interactions = interactions.interaction_df.query('user_id not in @user_with_few_interactions')
ncf_interactions.to_parquet('../data/moscow_slice/ncf_orders.parquet')
print("ncf_interactions:", len(ncf_interactions))

ncf_valid_users = set(ncf_interactions['user_id'].unique())
print("ncf_uniq_users:", len(ncf_valid_users))
ncf_valid_chains = set(ncf_interactions['chain_id'].unique())
print("ncf_uniq_chains:", len(ncf_valid_chains))

ncf_interactions: 3143215
ncf_uniq_users: 263989
ncf_uniq_chains: 7122


In [59]:
val_df = pd.read_pickle('../data/test_VALID.pkl')
val_df = val_df[['customer_id', 'h3', 'chain_id']]
val_df = val_df.rename(columns={"customer_id": "user_id"})
val_df.user_id = val_df.user_id.astype(int)
print("initial:")
print("df, uniq_users, uniq_chains:", len(val_df), len(val_df.user_id.unique()), len(val_df.chain_id.unique()))

val_df = val_df.query('h3 in @h3index.valid')
print()
print("after invalid h3 filtering:")
print("df, uniq_users, uniq_chains:", len(val_df), len(val_df.user_id.unique()), len(val_df.chain_id.unique()))

val_df = val_df.query('user_id in @interactions.user_index')
print()
print("after invalid users filtering:")
print("df, uniq_users, uniq_chains:", len(val_df), len(val_df.user_id.unique()), len(val_df.chain_id.unique()))

val_df = val_df.query('chain_id in @interactions.chain_index')
print()
print("after invalid chains filtering:")
print("df, uniq_users, uniq_chains:", len(val_df), len(val_df.user_id.unique()), len(val_df.chain_id.unique()))

val_df = val_df.drop_duplicates()
print()
print("after dropping duplicates:")
print("df, uniq_users, uniq_chains:", len(val_df), len(val_df.user_id.unique()), len(val_df.chain_id.unique()))

initial:
df, uniq_users, uniq_chains: 2300001 1253198 19810

after invalid h3 filtering:
df, uniq_users, uniq_chains: 2293762 1249258 19788

after invalid users filtering:
df, uniq_users, uniq_chains: 483559 212160 11404

after invalid chains filtering:
df, uniq_users, uniq_chains: 341373 172590 5297

after dropping duplicates:
df, uniq_users, uniq_chains: 256964 172590 5297


In [80]:
val_df.head()

,user_id,h3,chain_id
0,386249,8911aa7aa1bffff,29454
2,59217420,8911aa4c867ffff,45822
11,0,8911aa70a97ffff,28720
14,8980391,8911aa09b4bffff,53904
19,8298875,8911aa70b2fffff,29454


In [98]:
# prepare for ncf
ncf_val_df = pd.DataFrame()
for user_id in tqdm(val_df.user_id.unique()):
    user_data = pd.DataFrame(val_df[val_df['user_id'] == user_id])
    user_data['label'] = 1 # presented in val_df
    other_data_lst = [ncf_val_df, user_data]
    for user_h3 in user_data['h3'].unique():
        user_chains = set(user_data[user_data['h3'] == h3].chain_id.unique())
        other_chains = set(h3index.h3_to_chains[user_h3])
        other_chains = other_chains.difference(user_chains)
        other_data = pd.DataFrame(other_chains, columns=['chain_id'])
        other_data['user_id'] = user_id
        other_data['h3'] = user_h3
        other_data['label'] = 0 # not presented in val_df
        other_data_lst.append(other_data)
    ncf_val_df = pd.concat(other_data_lst, ignore_index=True, sort=False)
print("df, uniq_users, uniq_chains:", len(ncf_val_df), len(ncf_val_df.user_id.unique()), len(ncf_val_df.chain_id.unique()))
ncf_val_df.to_parquet('../data/moscow_slice/ncf_pred_interactions.parquet')

  0%|          | 691/172590 [04:32<18:50:20,  2.53it/s]


KeyboardInterrupt: 

## Сколько данных тестовой выборки отсеивается в зависимости от interactions

### Clicks + orders: full

initial:
df, uniq_users, uniq_chains: 2300001 1253198 19810

after invalid h3 filtering:
df, uniq_users, uniq_chains: 2293762 1249258 19788

after invalid users filtering:
df, uniq_users, uniq_chains: 1987082 1044374 19453

after invalid chains filtering:
df, uniq_users, uniq_chains: 1984220 1043382 19118


### Orders: full

initial:
df, uniq_users, uniq_chains: 2300001 1253198 19810

after invalid h3 filtering:
df, uniq_users, uniq_chains: 2293762 1249258 19788

after invalid users filtering:
df, uniq_users, uniq_chains: 1860055 952741 19285

after invalid chains filtering:
df, uniq_users, uniq_chains: 1856314 951570 18664

### Clicks + orders: processed full

initial:
df, uniq_users, uniq_chains: 2300001 1253198 19810

after invalid h3 filtering:
df, uniq_users, uniq_chains: 2293762 1249258 19788

after invalid users filtering:
df, uniq_users, uniq_chains: 851440 415267 17272

after invalid chains filtering:
df, uniq_users, uniq_chains: 692249 369776 16446

### Orders: processed moscow

initial:
df, uniq_users, uniq_chains: 2300001 1253198 19810

after invalid h3 filtering:
df, uniq_users, uniq_chains: 2293762 1249258 19788

after invalid users filtering:
df, uniq_users, uniq_chains: 483559 212160 11404

after invalid chains filtering:
df, uniq_users, uniq_chains: 341373 172590 5297

### Orders: processed saint-peterburg
initial:
df, uniq_users, uniq_chains: 2300001 1253198 19810

after invalid h3 filtering:
df, uniq_users, uniq_chains: 2293762 1249258 19788

after invalid users filtering:
df, uniq_users, uniq_chains: 164699 62373 5801

after invalid chains filtering:
df, uniq_users, uniq_chains: 87146 46513 1384

### Orders: processed other regions

initial:
df, uniq_users, uniq_chains: 2300001 1253198 19810

after invalid h3 filtering:
df, uniq_users, uniq_chains: 2293762 1249258 19788

after invalid users filtering:
df, uniq_users, uniq_chains: 376063 178911 13700

after invalid chains filtering:
df, uniq_users, uniq_chains: 240902 136697 9381

# Выводы:
* 20% тестовых юзеров нет ни в clicks, ни в orders (cold start);
* 75% тестовых юзеров есть в orders, т.е clicks можно не рассматривать (всего 5%);
* только 30% (!!!!) тестовых юзеров остается после вызова processed_data;
* 48% -- москва, 39% -- регионы, 13% -- спб в orders после вызова processed_data;
* также есть сделать val.drop_duplicates, то отсортируется порядка 30% строк!

### Если h3 пользователя неизвестен, то можно брать следующий в иерархии h3 (более крупный)

In [ ]:
val_df = pd.pivot_table(val_df,
                        values=['chain_id'],
                        index=['user_id', 'h3'],
                        aggfunc={'chain_id': set})
val_df = val_df.reset_index()

In [ ]:
def predict(model, user_id, h3, thr=0.9, top_k=10, filter_already_liked_items=True):
    user_index = interactions.user_index[user_id]
    valid_chains = h3index.h3_to_chains[h3]
    filter_items = [v for k, v in interactions.chain_index.items() if k not in valid_chains]
    top = model.recommend(user_index,
                          interactions.sparse_interaction_matrix.T,
                          N=top_k,
                          filter_already_liked_items=filter_already_liked_items,
                          filter_items=filter_items)
    top = [interactions.r_chain_index[x] for x, score in top if score > thr]
    return top

def old_items(user_id):
    return set(interactions.interaction_df[interactions.interaction_df['user_id'] == user_id]['chain_id'].unique())

In [ ]:
def metric(y_true, y_pred, y_old, at1=10, at2=30, average=True):
    """
    new_prec@10 + new_prec@30 + 1/2 *(prec_@10 + prec@30)
    """
    scores_new = []
    scores_all = []
    scores_total = []
    for t, p, o in zip(y_true, y_pred, y_old):
        t = list(t)
        p = list(p)
        o = o if isinstance(o, (set, list)) else []
        
        prec1 = len(set(t[:at1]) & set(p[:at1])) / at1
        prec2 = len(set(t[:at2]) & set(p[:at2])) / at2
        new_prec1 = len((set(p[:at1]) - set(o)) & set(t[:at1])) / at1
        new_prec2 = len((set(p[:at2]) - set(o)) & set(t[:at2])) / at2

        scores_total.append(new_prec1 + new_prec2 + 0.5 * (prec1 + prec2))
        scores_new.append(new_prec1 + new_prec2)
        scores_all.append(prec1 + prec2)

    return (np.mean(scores_total) if average else scores_total,
            np.mean(scores_new) if average else scores_new,
            np.mean(scores_all) if average else scores_all)

In [ ]:
# !pip install implicit
import implicit

def hyper_params(val_df, factors=60, thr=0.7, top_k=30, filter_liked=True):
    print('factors: ', factors, ', thr: ', thr, ', top_k: ', top_k, ', filter_liked: ', filter_liked)
    model = implicit.als.AlternatingLeastSquares(factors=factors)
    model.fit(interactions.sparse_interaction_matrix)
    val = val_df
    val['pred_chains'] = val.apply(lambda x: predict(model, x.user_id, x.h3, thr, top_k, filter_liked), axis=1)
    val['old_chains'] = val.apply(lambda x: old_items(x.user_id), axis=1)
    scores = metric(val['chain_id'], val['pred_chains'], val['old_chains'])
    print('total, new, all = ', scores)
    print()

In [ ]:
hyper_params(val_df, factors=60, thr=0.7, top_k=30, filter_liked=True)

factors:  60 , thr:  0.7 , top_k:  30 , filter_liked:  True

total, new, all =  (0.02605082142811052, 0.00023745918670228555, 0.05162672448281647)

In [ ]:
for factors in [30, 40, 50, 60, 70]:
    for thr in [0.7, 0.75, 0.8, 0.85, 0.9]:
        for top_k in [5, 10, 20, 30]:
            for filter_liked in [True, False]:
               hyper_params(val_df, factors, thr, top_k, filter_liked) 